In [5]:
using JuMP, Cbc
o = Model(with_optimizer(Cbc.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

In [9]:
#viewers in millions in hours 9:21
v = [281.7
422
423
376.8
377.8
518.1
528
529
549.6
858.5
859.5
860.5
861.5]

13-element Array{Float64,1}:
 281.7
 422.0
 423.0
 376.8
 377.8
 518.1
 528.0
 529.0
 549.6
 858.5
 859.5
 860.5
 861.5

In [10]:
# qualifications variables
@variable(o, m_qual1[1:2, 9:21], Bin)      #men's qualification subdivision 1
@variable(o, m_qual2[1:2, 9:21], Bin)      #men's qualification subdivision 2
@variable(o, m_qual3[1:2, 9:21], Bin)      #men's qualification subdivision 3
@variable(o, w_qual1[1:2, 9:21], Bin)      #women's qualification subdivision 1
@variable(o, w_qual2[1:2, 9:21], Bin)      #women's qualification subdivision 2
@variable(o, w_qual3[1:2, 9:21], Bin)      #women's qualification subdivision 3

mens_qual = [m_qual1, m_qual2, m_qual3]
womens_qual = [w_qual1, w_qual2, w_qual3]
all_qual = [m_qual1, m_qual2, m_qual3, w_qual1, w_qual2, w_qual3]


# qualifications constraints
# all mens qual rounds on either day 1 or 2 all womens qual rounds on either day 1 or 2
@variable(o, w, Bin)    
@constraint(o, sum(m_qual1[1]+m_qual2[1]+m_qual3[1]) == 6-(6*w))
@constraint(o, sum(m_qual1[2]+m_qual2[2]+m_qual3[2]) == 6*w)
@constraint(o, sum(w_qual1[1]+w_qual2[1]+w_qual3[1]) == 6*w)
@constraint(o, sum(w_qual1[2]+w_qual2[2]+w_qual3[2]) == 6-(6*w))


#@constraint(o, sum(m_qual1[1, i] for i in 9:12) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 11:14) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 12:15) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 13:16) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 14:17) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 15:18) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 16:19) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 17:20) <= 2)
#@constraint(o, sum(m_qual1[1, i] for i in 18:21) <= 2)

#For each (m_qual2&3, w_qual1,2&3)??


# each session takes 2 hours
#Idk how to make sure the hours are 2 in a row 

ErrorException: An object of name m_qual1 is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

In [11]:
#each event takes two hours and there are 2 hrs between events
@variable(o, h, Bin) #auxiliarry variable h that is equal to 1 if an event starts at hour h
for event in all_qual
    for day in 1:2
        for hour in 9:18
            @constraint(o, event[day, hour] + event[day, hour + 1] <= 2*h)    #each event takes two hours
            @constraint(o, event[day, hour + 2] + event[day, hour + 3] <= h)  #there are 2 hrsbetween the qualifiers
        end
    end
end

ErrorException: An object of name h is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

In [12]:
o

A JuMP Model
Feasibility problem with:
Variables: 158
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 240 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 158 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: h, m_qual1, m_qual2, m_qual3, w, w_qual1, w_qual2, w_qual3

In [13]:
# team finals
@variable(o, m_tf[4:5, 9:21], Bin)
@variable(o, w_tf[4:5, 9:21], Bin)

# 2 hr for men, 2 hrs for women on different days for women
#Same for this like how do we make sure the hours would be 9&10 vs 9&11


# men and women on different days
@variable(o, x, Bin)
@constraint(o, sum(m_tf[4, i] for i in 9:21) <= 2*x)
@constraint(o, sum(m_tf[5, i] for i in 9:21) <= 2-(2*x))
@constraint(o, sum(w_tf[4, i] for i in 9:21) <= 2-(2*x))
@constraint(o, sum(w_tf[5, i] for i in 9:21) <= 2*x

LoadError: syntax: incomplete: premature end of input

In [14]:
# individual all around
@variable(o, m_ar[6:7, 9:21])
@variable(o, w_ar[6:7, 9:21])

# individual constraints
#2 hrs for women, 3 hrs for men


# men and women on different days
@variable(o, t, Bin)
@constraint(o, sum(m_ar[6, i] for i in 9:21) <= 3*t)
@constraint(o, sum(m_ar[7, i] for i in 9:21) <= 3-(3*t))
@constraint(o, sum(w_ar[6, i] for i in 9:21) <= 2-(2*t))
@constraint(o, sum(w_ar[7, i] for i in 9:21) <= 2*t)

w_ar[7,9] + w_ar[7,10] + w_ar[7,11] + w_ar[7,12] + w_ar[7,13] + w_ar[7,14] + w_ar[7,15] + w_ar[7,16] + w_ar[7,17] + w_ar[7,18] + w_ar[7,19] + w_ar[7,20] + w_ar[7,21] - 2 t ≤ 0.0

In [17]:
# specific events
@variable(o, m_floor[9:11, 9:21])      # mens floor
@variable(o, m_vault[9:11, 9:21])      # mens vault
@variable(o, m_pb[9:11, 9:21])         # mens parallel bars
@variable(o, m_pommel[9:11, 9:21])     # mens pommel horse
@variable(o, m_rings[9:11, 9:21])      # mens rings
@variable(o, m_horizontal[9:11, 9:21]) #mens horizontal bar
@variable(o, w_floor[9:11, 9:21])      # womens floor
@variable(o, w_vault[9:11, 9:21])      # womens vault
@variable(o, w_ub[9:11, 9:21])         # womens uneven bars
@variable(o, w_b[9:11, 9:21])          # womens beam


@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) <=4)


#at most two specific events can happen in an hour
#for each day of specific events, the sum of events in an hour cannot be more than 2
for i = 9:21
    @constraint(o, (m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i]) <=2)
    end

#if womens floor is happening, mens floor cannot be happening (there is only one floor)
for i = 9:11
    for j = 9:21
        @constraint(o, (m_floor[i,j]+ w_floor[i,j) <=1))
    end
end




ErrorException: An object of name m_floor is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

In [ ]:
#add up viewers at possible times of events to get objective to be maximized
#mens qual
objec = v'*(m_qual1[1,:])+v'*(m_qual1[2,:])+ v'*(m_qual2[1,:])+v'*(m_qual2[2,:])+ v'*(m_qual3[1,:])+v'*(m_qual3[2,:])
#womens qual
objec = objec + v'*(w_qual1[1,:])+v'*(w_qual1[2,:])+ v'*(w_qual2[1,:])+v'*(w_qual2[2,:])+ v'*(w_qual3[1,:])+v'*(w_qual3[2,:])
#team finals
objec = objec +v'*m_tf[4,:]+v'*m_tf[5,:]+v'*w_tf[4,:]+v'*w_tf[5,:]
#indiv all around
objec = objec +v'*m_ar[6,:]+v'*m_ar[7,:]+v'*w_ar[6,:]+v'*w_ar[7,:]
#specific events
#always a day of rest before finals (day 8)
objec = objec +sum(v'*m_floor[i,:] for i in 9:11)
objec = objec +sum(v'*m_vault[i,:] for i in 9:11)
objec = objec +sum(v'*m_pb[i,:] for i in 9:11)
objec = objec +sum(v'*m_pommel[i,:] for i in 9:11)
objec = objec +sum(v'*m_horizontal[i,:] for i in 9:11)
objec = objec +sum(v'*m_rings[i,:] for i in 9:11)
objec = objec +sum(v'*w_floor[i,:] for i in 9:11)
objec = objec +sum(v'*w_vault[i,:] for i in 9:11)
objec = objec +sum(v'*w_ub[i,:] for i in 9:11)
objec = objec +sum(v'*w_b[i,:] for i in 9:11)


#objective

#@objective(o, Max,  objec)


#take a look at our model
#o

#optimize!(o)